In [16]:
import bibtexparser
from rapidfuzz import fuzz
from rapidfuzz import process
import requests
import xml.etree.ElementTree as ET
import time
import re 

In [ ]:
import urllib.parse
import urllib.request
import time
import feedparser

base_url = 'http://export.arxiv.org/api/query?'

search_query = (
    'all:("llm" OR "language model") AND ("se" OR "software engineer" OR '
    '"software develop" OR "code" OR "coding" OR "program" OR "requirement" OR "bug" OR "test" OR '
    '"debug" OR "refactor" OR "vulnerab" OR "fault" OR "clon")'
)

all_links = []
start = 0
results_per_iteration = 5
wait_time = 3

params = {
    'search_query': search_query,
    'start': start,
    'max_results': results_per_iteration
}
query = urllib.parse.urlencode(params)
url = base_url + query
response = urllib.request.urlopen(url).read()
feed = feedparser.parse(response)

# total_results = int(feed.feed.opensearch_totalresults)
total_results = 10  # Placeholder for total results, adjust as needed
print("Total results found:", total_results)

for i in range(0, total_results, results_per_iteration):
    params['start'] = i
    query = urllib.parse.urlencode(params)
    url = base_url + query

    response = urllib.request.urlopen(url).read()
    feed = feedparser.parse(response)

    for entry in feed.entries:   
        all_links.append(entry.links[1].href)

    print(f'Sleeping for {wait_time} seconds')
    time.sleep(wait_time)


Total results found: 10
Sleeping for 3 seconds
Sleeping for 3 seconds


In [ ]:
import urllib.request
import fitz  
import re

def extract_emails_from_pdf_url(pdf_url):
    try:
        # Download PDF into memory
        response = urllib.request.urlopen(pdf_url)
        pdf_data = response.read()
        
        # Open PDF from bytes
        pdf_document = fitz.open(stream=pdf_data, filetype="pdf")
        
        full_text = ""
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            full_text += page.get_text()
        
        # Regex to extract normal emails
        emails = set(re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', full_text))
        
        # Regex to extract alias groups like {user1,user2}@domain.com
        alias_pattern = re.compile(r'\{([^\}]+)\}@([a-zA-Z0-9.-]+\.[a-zA-Z]{2,})')
        for match in alias_pattern.finditer(full_text):
            users = match.group(1)
            domain = match.group(2)
            usernames = [u.strip() for u in users.split(',')]
            for u in usernames:
                emails.add(f"{u}@{domain}")
        
        return emails

    except Exception as e:
        print(f"Failed to extract from {pdf_url}: {e}")
        return set()
    
all_emails = set()
batch_size = 10
waiting_time = 3  
for i in range(0, len(all_links), batch_size):
    batch = all_links[i:i + batch_size]
    for pdf_url in batch:
        emails = extract_emails_from_pdf_url(pdf_url)
        all_emails.update(emails)
    time.sleep(waiting_time) 
    # print("All emails found so far:", all_emails)


All emails found so far: {'mika.saari@tuni.fi', 'shahbaz.siddeeq@tuni.fi', 'muhammad.waseem@tuni.fi', 'henri.terho@eficode.com', 'mdmahade.hasan@tuni.fi', 'Kai-Kristian.Kemell@tuni.fi', 'zeeshan.rasheed@tuni.fi', 'kalle.makela@gmail.com', 'pekka.abrahamsson@tuni.fi', 'jussi.rasku@tuni.fi'}
All emails found so far: {'mika.saari@tuni.fi', 'shahbaz.siddeeq@tuni.fi', 'muhammad.waseem@tuni.fi', 'henri.terho@eficode.com', 'mdmahade.hasan@tuni.fi', 'Kai-Kristian.Kemell@tuni.fi', 'zeeshan.rasheed@tuni.fi', 'kalle.makela@gmail.com', 'rubenm@cs.cmu.edu', 'vhellendoorn@cmu.edu', 'pekka.abrahamsson@tuni.fi', 'clegoues@cs.cmu.edu', 'aidan@cmu.edu', 'jussi.rasku@tuni.fi'}
All emails found so far: {'mika.saari@tuni.fi', 'shahbaz.siddeeq@tuni.fi', 'muhammad.waseem@tuni.fi', 'henri.terho@eficode.com', 'btaghiz@ncsu.edu', 'mdmahade.hasan@tuni.fi', 'Kai-Kristian.Kemell@tuni.fi', 'zeeshan.rasheed@tuni.fi', 'kalle.makela@gmail.com', 'rubenm@cs.cmu.edu', 'vhellendoorn@cmu.edu', 'pekka.abrahamsson@tuni.fi', 

In [ ]:
import pickle

with open("emails.pkl", "wb") as f:
    pickle.dump(emails, f)

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(sender_email, sender_password, recipient_email, subject, body):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587) 
        server.starttls() 
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient_email, msg.as_string())
        server.quit()
        print(f"Email sent to {recipient_email}")
    except Exception as e:
        print(f"Failed to send email: {e}")

sender = "your_email@gmail.com"
password = "your_password_or_app_password"
recipient = "recipient@example.com"
subject = "Test Email"
body = "Hello! This is an automatic email."

send_email(sender, password, recipient, subject, body)
